In [ ]:
from fastai.vision.all import *
import torchaudio

In [ ]:
DATA_PATH = Path('/kaggle/input/rfcx-species-audio-detection/')
train_df = pd.read_csv(DATA_PATH/'train_tp.csv')
train_files = get_files(DATA_PATH/'train', '.flac')

In [ ]:
tfms = [torchaudio.transforms.MelSpectrogram(n_mels=32), torchaudio.transforms.AmplitudeToDB]
dls = DataBlock(blocks=(TransformBlock(type_tfms=tfms), MultiCategoryBlock),                   
                 get_x=lambda x: torchaudio.load(x)[0],
                 get_y=lambda x: set(train_df[train_df.recording_id == x.stem].species_id) or {24}).dataloaders(train_files, bs=32)

In [ ]:
learn = cnn_learner(dls, resnet18, config={"n_in":1}, metrics=[accuracy_multi])
learn.fine_tune(5, 1e-2)

In [ ]:
sample = pd.read_csv(DATA_PATH/'sample_submission.csv')
test_files = [DATA_PATH/'test'/f'{id}.flac' for id in sample.recording_id]

test_dl = dls.test_dl(test_files)
preds,_ = learn.get_preds(dl=test_dl)

submission = pd.concat([sample['recording_id'], pd.DataFrame(preds[:,:-1].numpy(), columns=sample.columns[1:])], axis=1)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)